In [1]:
import pandas as pd
import numpy as np

import datetime
import json
import requests
import re
from math import sin, cos, sqrt, atan2, radians


from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import IntProgress
from IPython.display import display
from IPython.display import Image
from IPython.core.display import HTML



from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

## Functions 

In [2]:
def get_distance(lat1,lon1,lat2,lon2):    
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians( lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c

In [3]:
def road_type(x):
    s= re.search(r"(rue|boulevard|avenue|chemin|place|terrasse|impasse|promenade|croissant|carré|côte|montée|allée)", x)
    try:
        return s.group()
    except :
        return "other"   

In [4]:
def trace_values(x,y,desc):
    fig = go.Figure()


    fig.add_trace(go.Scatter(x=x, y=y, name=desc,
                             line=dict(color='firebrick', width=2)))

    fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=1, label="2y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
    )

    # Edit the layout
    fig.update_layout(title='Trace Values {}'.format(desc),
                       xaxis_title='Date',
                       yaxis_title='Value')
    fig.show()
    return fig

## Read Data

In [5]:
df_remorquages_mtl = pd.read_csv("./remorquages.csv")

## Data Assessement

In [6]:
df_remorquages_mtl.head()

,DATE_ORIGINE,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,RUE_ORIGINE,SECTEUR_ORIGINE,ARRONDISSEMENT_ORIGINE,DATE_DESTINATION,LONGITUDE_DESTINATION,LATITUDE_DESTINATION,RUE_DESTINATION,SECTEUR_DESTINATION,ARRONDISSEMENT_DESTINATION,MOTIF_REMORQUAGE
0,2015-12-31T00:23:00,-73.587203,45.524981,rue Drolet,NaN,Plateau-Mont-Royal,2015-12-31T00:25:00,-73.586791,45.525131,rue de Grand-Pré,NaN,Plateau-Mont-Royal,Constat d'infraction
1,2015-12-31T00:24:00,-73.622256,45.585753,boulevard des Grandes-Prairies,NaN,Saint-Léonard,2015-12-31T00:30:00,-73.621005,45.586975,boulevard des Grandes-Prairies,NaN,Saint-Léonard,Constat d'infraction
2,2015-12-31T00:25:00,-73.600692,45.535027,rue Saint-Hubert,NaN,Rosemont - La Petite-Patrie,2015-12-31T00:32:00,-73.600881,45.535163,rue Saint-Hubert,NaN,Rosemont - La Petite-Patrie,Constat d'infraction
3,2015-12-31T00:25:00,-73.620824,45.516675,avenue Ducharme,NaN,Outremont,2015-12-31T00:28:00,-73.621995,45.518764,avenue Rockland,NaN,Outremont,Constat d'infraction
4,2015-12-31T00:27:00,-73.580197,45.494309,rue Saint-Mathieu,NaN,Ville-Marie,2015-12-31T00:39:00,-73.579972,45.493133,rue Sainte-Catherine Ouest,NaN,Ville-Marie,Constat d'infraction


In [7]:
df_remorquages_mtl.describe()

,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,SECTEUR_ORIGINE,LONGITUDE_DESTINATION,LATITUDE_DESTINATION,SECTEUR_DESTINATION
count,237683.000000,237683.000000,0.0,237533.000000,237533.000000,0.0
mean,-73.598093,45.526252,NaN,-73.598119,45.526283,NaN
std,0.042392,0.048360,NaN,0.042460,0.048385,NaN
min,-73.868060,45.415243,NaN,-73.867954,45.415282,NaN
25%,-73.625693,45.495262,NaN,-73.625718,45.495081,NaN
50%,-73.592951,45.527790,NaN,-73.592906,45.527954,NaN
75%,-73.568442,45.553061,NaN,-73.568133,45.553170,NaN
max,-73.480045,45.701455,NaN,-73.480730,45.701176,NaN


In [8]:
df_remorquages_mtl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238378 entries, 0 to 238377
Data columns (total 13 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   DATE_ORIGINE                238378 non-null  object 
 1   LONGITUDE_ORIGINE           237683 non-null  float64
 2   LATITUDE_ORIGINE            237683 non-null  float64
 3   RUE_ORIGINE                 238378 non-null  object 
 4   SECTEUR_ORIGINE             0 non-null       float64
 5   ARRONDISSEMENT_ORIGINE      237683 non-null  object 
 6   DATE_DESTINATION            238375 non-null  object 
 7   LONGITUDE_DESTINATION       237533 non-null  float64
 8   LATITUDE_DESTINATION        237533 non-null  float64
 9   RUE_DESTINATION             238378 non-null  object 
 10  SECTEUR_DESTINATION         0 non-null       float64
 11  ARRONDISSEMENT_DESTINATION  237439 non-null  object 
 12  MOTIF_REMORQUAGE            238217 non-null  object 
dtypes: float64(6),

> Replace : DATE_ORIGINE,DATE_DESTINATION to have datetime

In [9]:
df_remorquages_mtl.isnull().sum()

DATE_ORIGINE                       0
LONGITUDE_ORIGINE                695
LATITUDE_ORIGINE                 695
RUE_ORIGINE                        0
SECTEUR_ORIGINE               238378
ARRONDISSEMENT_ORIGINE           695
DATE_DESTINATION                   3
LONGITUDE_DESTINATION            845
LATITUDE_DESTINATION             845
RUE_DESTINATION                    0
SECTEUR_DESTINATION           238378
ARRONDISSEMENT_DESTINATION       939
MOTIF_REMORQUAGE                 161
dtype: int64

> Remove : SECTEUR_DESTINATION,SECTEUR_ORIGINE

> Assess : DATE_DESTINATION,ARRONDISSEMENT_ORIGINE,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,LONGITUDE_DESTINATION ,LATITUDE_DESTINATION,ARRONDISSEMENT_DESTINATION

In [10]:
#We have 693 null values
df_remorquages_mtl[df_remorquages_mtl["ARRONDISSEMENT_ORIGINE"].isnull()].head()

,DATE_ORIGINE,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,RUE_ORIGINE,SECTEUR_ORIGINE,ARRONDISSEMENT_ORIGINE,DATE_DESTINATION,LONGITUDE_DESTINATION,LATITUDE_DESTINATION,RUE_DESTINATION,SECTEUR_DESTINATION,ARRONDISSEMENT_DESTINATION,MOTIF_REMORQUAGE
756,2015-12-30T10:32:00,NaN,NaN,rue Jean-Bouillet,NaN,NaN,2015-12-30T10:37:00,NaN,NaN,rue Jean-Bouillet,NaN,NaN,Constat d'infraction
890,2015-12-30T07:00:00,NaN,NaN,avenue Baldwin,NaN,NaN,2015-12-30T07:10:00,-73.547961,45.609625,avenue Chaumont,NaN,Anjou,Constat d'infraction
891,2015-12-30T07:08:00,NaN,NaN,avenue Chaumont,NaN,NaN,2015-12-30T07:18:00,-73.547439,45.613740,avenue Lévesque,NaN,Anjou,Constat d'infraction
903,2015-12-30T08:45:00,NaN,NaN,avenue Jeanne-d'Arc,NaN,NaN,2015-12-30T08:50:00,-73.538586,45.549391,boulevard Pie-IX,NaN,Mercier - Hochelaga-Maisonneuve,Constat d'infraction
1063,2015-12-30T12:20:00,NaN,NaN,rue Filiatrault,NaN,NaN,2015-12-30T12:20:00,-73.681256,45.510808,rue Cardinal,NaN,Saint-Laurent,Constat d'infraction


In [11]:
#We have 693 null values
df_remorquages_mtl[df_remorquages_mtl["LONGITUDE_ORIGINE"].isnull()].head()

,DATE_ORIGINE,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,RUE_ORIGINE,SECTEUR_ORIGINE,ARRONDISSEMENT_ORIGINE,DATE_DESTINATION,LONGITUDE_DESTINATION,LATITUDE_DESTINATION,RUE_DESTINATION,SECTEUR_DESTINATION,ARRONDISSEMENT_DESTINATION,MOTIF_REMORQUAGE
756,2015-12-30T10:32:00,NaN,NaN,rue Jean-Bouillet,NaN,NaN,2015-12-30T10:37:00,NaN,NaN,rue Jean-Bouillet,NaN,NaN,Constat d'infraction
890,2015-12-30T07:00:00,NaN,NaN,avenue Baldwin,NaN,NaN,2015-12-30T07:10:00,-73.547961,45.609625,avenue Chaumont,NaN,Anjou,Constat d'infraction
891,2015-12-30T07:08:00,NaN,NaN,avenue Chaumont,NaN,NaN,2015-12-30T07:18:00,-73.547439,45.613740,avenue Lévesque,NaN,Anjou,Constat d'infraction
903,2015-12-30T08:45:00,NaN,NaN,avenue Jeanne-d'Arc,NaN,NaN,2015-12-30T08:50:00,-73.538586,45.549391,boulevard Pie-IX,NaN,Mercier - Hochelaga-Maisonneuve,Constat d'infraction
1063,2015-12-30T12:20:00,NaN,NaN,rue Filiatrault,NaN,NaN,2015-12-30T12:20:00,-73.681256,45.510808,rue Cardinal,NaN,Saint-Laurent,Constat d'infraction


In [12]:
#We have 3 null values
df_remorquages_mtl[df_remorquages_mtl["DATE_DESTINATION"].isnull()].head()

,DATE_ORIGINE,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,RUE_ORIGINE,SECTEUR_ORIGINE,ARRONDISSEMENT_ORIGINE,DATE_DESTINATION,LONGITUDE_DESTINATION,LATITUDE_DESTINATION,RUE_DESTINATION,SECTEUR_DESTINATION,ARRONDISSEMENT_DESTINATION,MOTIF_REMORQUAGE
8260,2016-01-21T20:12:00,-73.586258,45.491157,avenue Lincoln,NaN,Ville-Marie,NaN,NaN,NaN,rue Chomedey,NaN,NaN,Constat d'infraction
14514,2016-01-26T16:45:00,-73.563905,45.506396,boulevard René-Lévesque Ouest,NaN,Ville-Marie,NaN,NaN,NaN,rue De Bleury,NaN,NaN,NaN
81134,2017-12-17T00:52:00,-73.534813,45.550956,avenue Letourneux,NaN,Mercier - Hochelaga-Maisonneuve,NaN,NaN,NaN,rue Hochelaga,NaN,NaN,Constat d'infraction


In [13]:
#We have 845 null values
df_remorquages_mtl[df_remorquages_mtl["LONGITUDE_DESTINATION"].isnull()].sample(10)

,DATE_ORIGINE,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,RUE_ORIGINE,SECTEUR_ORIGINE,ARRONDISSEMENT_ORIGINE,DATE_DESTINATION,LONGITUDE_DESTINATION,LATITUDE_DESTINATION,RUE_DESTINATION,SECTEUR_DESTINATION,ARRONDISSEMENT_DESTINATION,MOTIF_REMORQUAGE
5018,2016-01-03T22:05:00,NaN,NaN,rue Bernard Est,NaN,NaN,2016-01-02T07:30:00,NaN,NaN,rue Saint-Dominique,NaN,NaN,Constat d'infraction
68004,2017-09-24T00:31:00,-73.573315,45.527099,rue De Brébeuf,NaN,Plateau-Mont-Royal,2017-09-24T00:31:00,NaN,NaN,rue Berri,NaN,NaN,Constat d'infraction
3643,2016-01-03T13:29:00,-73.586972,45.528602,rue Saint-Hubert,NaN,Plateau-Mont-Royal,2016-01-03T13:29:00,NaN,NaN,rue Saint-Grégoire,NaN,NaN,Constat d'infraction
205700,2019-11-14T10:33:00,NaN,NaN,boulevard Décarie,NaN,NaN,2019-11-14T10:33:00,NaN,NaN,boulevard Décarie,NaN,NaN,Constat d'infraction
36469,2016-12-13T21:00:00,-73.586284,45.490358,avenue Atwater,NaN,Ville-Marie,2016-12-13T21:00:00,NaN,NaN,rue Sherbrooke Ouest,NaN,NaN,Autre
127968,2018-02-15T21:32:00,-73.616682,45.478586,avenue Northcliffe,NaN,Côte-des-Neiges - Notre-Dame-de-Grâce,2018-02-15T21:42:00,NaN,NaN,avenue Northcliffe,NaN,NaN,Constat d'infraction
8832,2016-01-20T22:25:00,-73.628318,45.495851,avenue Dupuis,NaN,Côte-des-Neiges - Notre-Dame-de-Grâce,2016-01-21T23:00:00,NaN,NaN,rue Jean-Brillant,NaN,NaN,Constat d'infraction
198071,2019-11-16T08:20:00,NaN,NaN,terrasse de Louisbourg,NaN,NaN,2019-11-16T08:20:00,NaN,NaN,terrasse de Louisbourg,NaN,NaN,Constat d'infraction
232305,2020-01-20T19:52:00,-73.635260,45.481871,avenue Macdonald,NaN,Côte-des-Neiges - Notre-Dame-de-Grâce,2020-01-20T19:52:00,NaN,NaN,avenue Macdonald,NaN,NaN,Constat d'infraction
14514,2016-01-26T16:45:00,-73.563905,45.506396,boulevard René-Lévesque Ouest,NaN,Ville-Marie,NaN,NaN,NaN,rue De Bleury,NaN,NaN,NaN


> The 693 missing values in ARRONDISSEMENT_ORIGINE will have the value in ARRONDISSEMENT_DESTINATION as the time between the DATE_ORIGINE and DATE_DESTINATION is not significatif.

> We will let ARRONDISSEMENT_ORIGINE,LONGITUDE_ORIGINE AS IS to not alterate the computations.  

## Data Cleaning

In [14]:
#Remove : SECTEUR_DESTINATION,SECTEUR_ORIGINE
df_remorquages_mtl = df_remorquages_mtl.drop(["SECTEUR_DESTINATION","SECTEUR_ORIGINE"],axis=1)

In [15]:
df_remorquages_mtl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238378 entries, 0 to 238377
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   DATE_ORIGINE                238378 non-null  object 
 1   LONGITUDE_ORIGINE           237683 non-null  float64
 2   LATITUDE_ORIGINE            237683 non-null  float64
 3   RUE_ORIGINE                 238378 non-null  object 
 4   ARRONDISSEMENT_ORIGINE      237683 non-null  object 
 5   DATE_DESTINATION            238375 non-null  object 
 6   LONGITUDE_DESTINATION       237533 non-null  float64
 7   LATITUDE_DESTINATION        237533 non-null  float64
 8   RUE_DESTINATION             238378 non-null  object 
 9   ARRONDISSEMENT_DESTINATION  237439 non-null  object 
 10  MOTIF_REMORQUAGE            238217 non-null  object 
dtypes: float64(4), object(7)
memory usage: 20.0+ MB


In [16]:
#Replace ARRONDISSEMENT_ORIGINE's null values by  the corresponding values in ARRONDISSEMENT_DESTINATION
df_remorquages_mtl.loc[df_remorquages_mtl["ARRONDISSEMENT_ORIGINE"].isnull(),("ARRONDISSEMENT_ORIGINE")] = df_remorquages_mtl[df_remorquages_mtl["ARRONDISSEMENT_ORIGINE"].isnull()]["ARRONDISSEMENT_DESTINATION"]

In [17]:
# We will let the 166 entries that we are not able to fill with ARRONDISSEMENT_DESTINATION
df_remorquages_mtl[df_remorquages_mtl["ARRONDISSEMENT_ORIGINE"].isnull()].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166 entries, 756 to 238170
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   DATE_ORIGINE                166 non-null    object 
 1   LONGITUDE_ORIGINE           0 non-null      float64
 2   LATITUDE_ORIGINE            0 non-null      float64
 3   RUE_ORIGINE                 166 non-null    object 
 4   ARRONDISSEMENT_ORIGINE      0 non-null      object 
 5   DATE_DESTINATION            166 non-null    object 
 6   LONGITUDE_DESTINATION       5 non-null      float64
 7   LATITUDE_DESTINATION        5 non-null      float64
 8   RUE_DESTINATION             166 non-null    object 
 9   ARRONDISSEMENT_DESTINATION  0 non-null      object 
 10  MOTIF_REMORQUAGE            160 non-null    object 
dtypes: float64(4), object(7)
memory usage: 15.6+ KB


In [18]:
#Get datetime type for DATE_ORIGINE and DATE_DESTINATION
df_remorquages_mtl["date_ori"] =  pd.to_datetime(df_remorquages_mtl["DATE_ORIGINE"],errors="coerce" )
df_remorquages_mtl["date_des"] =  pd.to_datetime(df_remorquages_mtl["DATE_DESTINATION"],errors="coerce")

In [19]:
df_remorquages_mtl.head()

,DATE_ORIGINE,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,RUE_ORIGINE,ARRONDISSEMENT_ORIGINE,DATE_DESTINATION,LONGITUDE_DESTINATION,LATITUDE_DESTINATION,RUE_DESTINATION,ARRONDISSEMENT_DESTINATION,MOTIF_REMORQUAGE,date_ori,date_des
0,2015-12-31T00:23:00,-73.587203,45.524981,rue Drolet,Plateau-Mont-Royal,2015-12-31T00:25:00,-73.586791,45.525131,rue de Grand-Pré,Plateau-Mont-Royal,Constat d'infraction,2015-12-31 00:23:00,2015-12-31 00:25:00
1,2015-12-31T00:24:00,-73.622256,45.585753,boulevard des Grandes-Prairies,Saint-Léonard,2015-12-31T00:30:00,-73.621005,45.586975,boulevard des Grandes-Prairies,Saint-Léonard,Constat d'infraction,2015-12-31 00:24:00,2015-12-31 00:30:00
2,2015-12-31T00:25:00,-73.600692,45.535027,rue Saint-Hubert,Rosemont - La Petite-Patrie,2015-12-31T00:32:00,-73.600881,45.535163,rue Saint-Hubert,Rosemont - La Petite-Patrie,Constat d'infraction,2015-12-31 00:25:00,2015-12-31 00:32:00
3,2015-12-31T00:25:00,-73.620824,45.516675,avenue Ducharme,Outremont,2015-12-31T00:28:00,-73.621995,45.518764,avenue Rockland,Outremont,Constat d'infraction,2015-12-31 00:25:00,2015-12-31 00:28:00
4,2015-12-31T00:27:00,-73.580197,45.494309,rue Saint-Mathieu,Ville-Marie,2015-12-31T00:39:00,-73.579972,45.493133,rue Sainte-Catherine Ouest,Ville-Marie,Constat d'infraction,2015-12-31 00:27:00,2015-12-31 00:39:00


## Feature Engineering

In [20]:
#extract the date
df_remorquages_mtl["date"] =  pd.to_datetime(df_remorquages_mtl["DATE_ORIGINE"]).dt.strftime('%Y-%m-%d')
df_remorquages_mtl["date"] =  pd.to_datetime(df_remorquages_mtl["date"])

df_remorquages_mtl["date"].isnull().sum()

0

In [21]:
#extract the year
df_remorquages_mtl["date_year"] =  pd.DatetimeIndex(df_remorquages_mtl["DATE_ORIGINE"]).year
df_remorquages_mtl["date_year"].isnull().sum()

0

In [22]:
#extract the yearMonth
df_remorquages_mtl["date_yearMonth"] =  pd.to_datetime(df_remorquages_mtl["DATE_ORIGINE"]).dt.strftime('%Y-%m')
df_remorquages_mtl["date_yearMonth"] =  pd.to_datetime(df_remorquages_mtl["date_yearMonth"])

df_remorquages_mtl["date_yearMonth"].isnull().sum()

0

In [23]:
#calculate the time it took 
df_remorquages_mtl["date_delta"] =  (df_remorquages_mtl["date_des"] - df_remorquages_mtl["date_ori"]) / np.timedelta64(1, 'm')
df_remorquages_mtl["date_delta"].isnull().sum()

5

In [24]:
# get seasons : https://stackoverflow.com/questions/60285557/extract-seasons-from-datetime-pandas

df_date_offset = (df_remorquages_mtl["date_ori"].dt.month*100 + df_remorquages_mtl["date_ori"].dt.day - 320)%1300

df_remorquages_mtl['season'] = pd.cut(df_date_offset, [-1, 300, 602, 900, 1300], 
                      labels=['spring', 'summer', 'autumn', 'winter'])

df_remorquages_mtl["season"].isnull().sum()

0

In [25]:
#add distance in KM
df_remorquages_mtl["distance"] = df_remorquages_mtl.apply(lambda x: get_distance(x['LATITUDE_ORIGINE'],x['LONGITUDE_ORIGINE'],x['LATITUDE_DESTINATION'],x['LONGITUDE_DESTINATION']),axis=1)
df_remorquages_mtl["season"].isnull().sum()

0

In [26]:
#add distance in KM from downtown (Montreal's downtown coordinate : 45.5035° N, -73.5685° W )

df_remorquages_mtl["distance_dtown"] = df_remorquages_mtl.apply(lambda x: get_distance(x['LATITUDE_ORIGINE'],x['LONGITUDE_ORIGINE'],45.5035,-73.5685),axis=1)
df_remorquages_mtl["season"].isnull().sum()

0

In [74]:
#Add Road Type
df_remorquages_mtl["road_type"] = df_remorquages_mtl["RUE_ORIGINE"].apply(lambda x: road_type(x))
df_remorquages_mtl.head()

,DATE_ORIGINE,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,RUE_ORIGINE,ARRONDISSEMENT_ORIGINE,DATE_DESTINATION,LONGITUDE_DESTINATION,LATITUDE_DESTINATION,RUE_DESTINATION,ARRONDISSEMENT_DESTINATION,...,date,date_year,date_yearMonth,date_delta,season,distance,distance_dtown,street_type,district_street,road_type
0,2015-12-31T00:23:00,-73.587203,45.524981,rue Drolet,Plateau-Mont-Royal,2015-12-31T00:25:00,-73.586791,45.525131,rue de Grand-Pré,Plateau-Mont-Royal,...,2015-12-31,2015,2015-12-01,2.0,winter,0.036217,2.798889,rue,Plateau-Mont-Royal_rue Drolet,rue
1,2015-12-31T00:24:00,-73.622256,45.585753,boulevard des Grandes-Prairies,Saint-Léonard,2015-12-31T00:30:00,-73.621005,45.586975,boulevard des Grandes-Prairies,Saint-Léonard,...,2015-12-31,2015,2015-12-01,6.0,winter,0.167196,10.061822,boulevard,Saint-Léonard_boulevard des Grandes-Prairies,boulevard
2,2015-12-31T00:25:00,-73.600692,45.535027,rue Saint-Hubert,Rosemont - La Petite-Patrie,2015-12-31T00:32:00,-73.600881,45.535163,rue Saint-Hubert,Rosemont - La Petite-Patrie,...,2015-12-31,2015,2015-12-01,7.0,winter,0.021071,4.311844,rue,Rosemont - La Petite-Patrie_rue Saint-Hubert,rue
3,2015-12-31T00:25:00,-73.620824,45.516675,avenue Ducharme,Outremont,2015-12-31T00:28:00,-73.621995,45.518764,avenue Rockland,Outremont,...,2015-12-31,2015,2015-12-01,3.0,winter,0.249612,4.333846,avenue,Outremont_avenue Ducharme,avenue
4,2015-12-31T00:27:00,-73.580197,45.494309,rue Saint-Mathieu,Ville-Marie,2015-12-31T00:39:00,-73.579972,45.493133,rue Sainte-Catherine Ouest,Ville-Marie,...,2015-12-31,2015,2015-12-01,12.0,winter,0.132003,1.369938,rue,Ville-Marie_rue Saint-Mathieu,rue


In [28]:
df_remorquages_mtl["district_street"]= df_remorquages_mtl.apply(lambda x: "{0}_{1}".format(x['ARRONDISSEMENT_ORIGINE'],x['RUE_ORIGINE']), axis=1)
df_remorquages_mtl.head()

,DATE_ORIGINE,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,RUE_ORIGINE,ARRONDISSEMENT_ORIGINE,DATE_DESTINATION,LONGITUDE_DESTINATION,LATITUDE_DESTINATION,RUE_DESTINATION,ARRONDISSEMENT_DESTINATION,...,date_des,date,date_year,date_yearMonth,date_delta,season,distance,distance_dtown,street_type,district_street
0,2015-12-31T00:23:00,-73.587203,45.524981,rue Drolet,Plateau-Mont-Royal,2015-12-31T00:25:00,-73.586791,45.525131,rue de Grand-Pré,Plateau-Mont-Royal,...,2015-12-31 00:25:00,2015-12-31,2015,2015-12-01,2.0,winter,0.036217,2.798889,rue,Plateau-Mont-Royal_rue Drolet
1,2015-12-31T00:24:00,-73.622256,45.585753,boulevard des Grandes-Prairies,Saint-Léonard,2015-12-31T00:30:00,-73.621005,45.586975,boulevard des Grandes-Prairies,Saint-Léonard,...,2015-12-31 00:30:00,2015-12-31,2015,2015-12-01,6.0,winter,0.167196,10.061822,boulevard,Saint-Léonard_boulevard des Grandes-Prairies
2,2015-12-31T00:25:00,-73.600692,45.535027,rue Saint-Hubert,Rosemont - La Petite-Patrie,2015-12-31T00:32:00,-73.600881,45.535163,rue Saint-Hubert,Rosemont - La Petite-Patrie,...,2015-12-31 00:32:00,2015-12-31,2015,2015-12-01,7.0,winter,0.021071,4.311844,rue,Rosemont - La Petite-Patrie_rue Saint-Hubert
3,2015-12-31T00:25:00,-73.620824,45.516675,avenue Ducharme,Outremont,2015-12-31T00:28:00,-73.621995,45.518764,avenue Rockland,Outremont,...,2015-12-31 00:28:00,2015-12-31,2015,2015-12-01,3.0,winter,0.249612,4.333846,avenue,Outremont_avenue Ducharme
4,2015-12-31T00:27:00,-73.580197,45.494309,rue Saint-Mathieu,Ville-Marie,2015-12-31T00:39:00,-73.579972,45.493133,rue Sainte-Catherine Ouest,Ville-Marie,...,2015-12-31 00:39:00,2015-12-31,2015,2015-12-01,12.0,winter,0.132003,1.369938,rue,Ville-Marie_rue Saint-Mathieu


In [29]:
#Get a sample of the Data Frame
df_remorquages_mtl.sample(10)

,DATE_ORIGINE,LONGITUDE_ORIGINE,LATITUDE_ORIGINE,RUE_ORIGINE,ARRONDISSEMENT_ORIGINE,DATE_DESTINATION,LONGITUDE_DESTINATION,LATITUDE_DESTINATION,RUE_DESTINATION,ARRONDISSEMENT_DESTINATION,...,date_des,date,date_year,date_yearMonth,date_delta,season,distance,distance_dtown,street_type,district_street
175343,2019-02-24T01:15:00,-73.607027,45.529548,boulevard Saint-Laurent,Rosemont - La Petite-Patrie,2019-02-24T01:21:00,-73.607630,45.529841,boulevard Saint-Laurent,Rosemont - La Petite-Patrie,...,2019-02-24 01:21:00,2019-02-24,2019,2019-02-01,6.0,winter,0.057146,4.172682,boulevard,Rosemont - La Petite-Patrie_boulevard Saint-La...
99579,2018-01-11T14:18:00,-73.572021,45.510631,rue Sainte-Famille,Plateau-Mont-Royal,2018-01-11T14:21:00,-73.576806,45.513052,avenue des Pins Ouest,Plateau-Mont-Royal,...,2018-01-11 14:21:00,2018-01-11,2018,2018-01-01,3.0,winter,0.460040,0.839339,rue,Plateau-Mont-Royal_rue Sainte-Famille
206377,2020-01-02T19:43:00,-73.627239,45.464545,rue Sherbrooke Ouest,Côte-des-Neiges - Notre-Dame-de-Grâce,2020-01-02T19:45:00,-73.625794,45.464524,avenue Benny,Côte-des-Neiges - Notre-Dame-de-Grâce,...,2020-01-02 19:45:00,2020-01-02,2020,2020-01-01,2.0,winter,0.112750,6.305357,rue,Côte-des-Neiges - Notre-Dame-de-Grâce_rue Sher...
172963,2019-02-25T19:05:00,-73.531333,45.603357,rue Baldwin,Mercier - Hochelaga-Maisonneuve,2019-02-25T19:07:00,-73.532849,45.603955,rue De Forbin-Janson,Mercier - Hochelaga-Maisonneuve,...,2019-02-25 19:07:00,2019-02-25,2019,2019-02-01,2.0,winter,0.135393,11.478080,rue,Mercier - Hochelaga-Maisonneuve_rue Baldwin
186738,2019-02-25T22:06:00,-73.588968,45.524243,avenue de l'Hôtel-de-Ville,Plateau-Mont-Royal,2019-02-25T22:11:00,-73.584141,45.522531,avenue du Mont-Royal Est,Plateau-Mont-Royal,...,2019-02-25 22:11:00,2019-02-25,2019,2019-02-01,5.0,winter,0.421602,2.805063,avenue,Plateau-Mont-Royal_avenue de l'Hôtel-de-Ville
52061,2017-01-04T20:45:00,-73.592692,45.520744,boulevard Saint-Joseph Ouest,Plateau-Mont-Royal,2017-01-04T20:50:00,-73.594652,45.523746,avenue Fairmount Ouest,Plateau-Mont-Royal,...,2017-01-04 20:50:00,2017-01-04,2017,2017-01-01,5.0,winter,0.367209,2.689712,boulevard,Plateau-Mont-Royal_boulevard Saint-Joseph Ouest
25980,2016-08-14T11:16:00,-73.558940,45.513044,rue Saint-Denis,Ville-Marie,2016-08-14T11:27:00,-73.553621,45.512377,rue Saint-Antoine Est,Ville-Marie,...,2016-08-14 11:27:00,2016-08-14,2016,2016-08-01,11.0,summer,0.421138,1.297022,rue,Ville-Marie_rue Saint-Denis
159805,2019-02-02T00:25:00,-73.618869,45.540290,rue Berri,Villeray - Saint-Michel - Parc-Extension,2019-02-02T00:27:00,-73.618014,45.538565,rue Drolet,Villeray - Saint-Michel - Parc-Extension,...,2019-02-02 00:27:00,2019-02-02,2019,2019-02-01,2.0,winter,0.203127,5.670507,rue,Villeray - Saint-Michel - Parc-Extension_rue B...
188971,2019-02-25T22:26:00,-73.539336,45.548727,avenue Jeanne-d'Arc,Mercier - Hochelaga-Maisonneuve,2019-02-25T22:28:00,-73.540403,45.548189,rue La Fontaine,Mercier - Hochelaga-Maisonneuve,...,2019-02-25 22:28:00,2019-02-25,2019,2019-02-01,2.0,winter,0.102433,5.520082,avenue,Mercier - Hochelaga-Maisonneuve_avenue Jeanne-...
95988,2018-01-18T12:41:00,-73.570626,45.460270,rue Rielle,Verdun,2018-01-18T12:43:00,-73.569004,45.460234,rue Rielle,Verdun,...,2018-01-18 12:43:00,2018-01-18,2018,2018-01-01,2.0,winter,0.126645,4.811364,rue,Verdun_rue Rielle


In [30]:
#Get a sample of the Data Frame
df_remorquages_mtl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238378 entries, 0 to 238377
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   DATE_ORIGINE                238378 non-null  object        
 1   LONGITUDE_ORIGINE           237683 non-null  float64       
 2   LATITUDE_ORIGINE            237683 non-null  float64       
 3   RUE_ORIGINE                 238378 non-null  object        
 4   ARRONDISSEMENT_ORIGINE      238212 non-null  object        
 5   DATE_DESTINATION            238375 non-null  object        
 6   LONGITUDE_DESTINATION       237533 non-null  float64       
 7   LATITUDE_DESTINATION        237533 non-null  float64       
 8   RUE_DESTINATION             238378 non-null  object        
 9   ARRONDISSEMENT_DESTINATION  237439 non-null  object        
 10  MOTIF_REMORQUAGE            238217 non-null  object        
 11  date_ori                    238378 non-

## Data Analysis

In [96]:
#Montreal's Districts
Image(url= "./ville_mtl.jpeg")

### What are the most  impacted districts ?

In [89]:
data = df_remorquages_mtl.groupby(["ARRONDISSEMENT_ORIGINE"]).\
                            count().\
                            sort_values("DATE_ORIGINE",ascending=True)
data = data.reset_index()      
fig = px.bar(data, x='DATE_ORIGINE', y='ARRONDISSEMENT_ORIGINE')
fig.update_layout(
    xaxis_title="Number of towing operation",
    yaxis_title="Boroughs",
)

fig.show()

#### Is it consistant over time ?

In [93]:
data = df_remorquages_mtl.groupby(["ARRONDISSEMENT_ORIGINE","date_year"])\
                        .agg({'DATE_ORIGINE':"count", 
                         'date_delta':'mean',
                         'distance':'mean',
                         'date_ori':'max', 
                         'date_des':'max'})\
                        .sort_values("DATE_ORIGINE",ascending=True).reset_index()

boroughs_top =  df_remorquages_mtl.groupby("ARRONDISSEMENT_ORIGINE")\
                                    .count()\
                                    .sort_values("DATE_ORIGINE",ascending=False)\
                                    .head(5).index


data = data[data["ARRONDISSEMENT_ORIGINE"].isin(boroughs_top)]
data = data.sort_values("date_year",ascending=True)
data = data[data.date_year>2015]

fig = go.Figure()
ci=0
for borough in boroughs_top:
    data_boroughs = data[data["ARRONDISSEMENT_ORIGINE"]== borough]
    fig.add_trace(go.Scatter(x=data_boroughs['date_year'], y=data_boroughs['DATE_ORIGINE'], name=borough,
                             line=dict(color=px.colors.qualitative.Dark2[ci], width=2)))
    ci= (ci+1) %8
    
    
fig.update_layout(
    xaxis_title="",
    yaxis_title="Number of towing operation",
    legend_title="Boroughs"
)    
fig.show()

### Are there streets that are more problematic ?

In [66]:
data = df_remorquages_mtl.groupby(["district_street"]).\
                            count().\
                            sort_values("DATE_ORIGINE",ascending=True)
data = data.reset_index()
fig = px.bar(data.tail(25), x='DATE_ORIGINE', y='district_street')
fig.show()

### In wich boroughs are located those streets ?

In [94]:
data = df_remorquages_mtl.groupby(["district_street","ARRONDISSEMENT_ORIGINE"]).\
                            count().\
                            sort_values("DATE_ORIGINE",ascending=False)


top_100 = data.head(100).reset_index()

top_100 = top_100.groupby(["ARRONDISSEMENT_ORIGINE"]).\
                            count().\
                            sort_values("DATE_ORIGINE").reset_index()

fig = px.bar(top_100, x='DATE_ORIGINE', y='ARRONDISSEMENT_ORIGINE')

    
fig.update_layout(
    xaxis_title="Number of streets",
    yaxis_title="Boroughs",
)   

fig.show()

In [95]:
data = df_remorquages_mtl.groupby(["district_street","ARRONDISSEMENT_ORIGINE"]).\
                            count().\
                            sort_values("DATE_ORIGINE",ascending=False)


top_500 = data.head(500).reset_index()

top_500 = top_500.groupby(["ARRONDISSEMENT_ORIGINE"]).\
                            count().\
                            sort_values("DATE_ORIGINE").reset_index()

fig = px.bar(top_500, x='DATE_ORIGINE', y='ARRONDISSEMENT_ORIGINE')

    
fig.update_layout(
    xaxis_title="Number of streets",
    yaxis_title="Boroughs",
)   

fig.show()

### What are the most impacted road type ?

In [77]:
data = df_remorquages_mtl.groupby(["road_type"])\
                        .count()\
                        .sort_values("DATE_ORIGINE",ascending=True).reset_index()
data = data[data["DATE_ORIGINE"]> 1000]
fig = px.pie(data, values='DATE_ORIGINE', names='road_type', title='Road Type')
fig.show()

### Does the season has an impact ?

In [79]:
data = df_remorquages_mtl.groupby(["season"])\
                        .agg({'DATE_ORIGINE':"count", 
                         'date_delta':'mean',
                         'distance':'mean',
                         'date_ori':'max', 
                         'date_des':'max'})\
                        .sort_values("DATE_ORIGINE",ascending=True).reset_index()

fig = px.pie(data, values='DATE_ORIGINE', names='season', title='Seasons')
fig.show()

In [78]:
data = df_remorquages_mtl.groupby(["date_year","date_yearMonth"])\
                        .agg({'DATE_ORIGINE':"count", 
                         'date_delta':'mean',
                         'distance':'mean',
                         'date_ori':'max', 
                         'date_des':'max'})\
                        .sort_values("DATE_ORIGINE",ascending=True).reset_index()

data = data.sort_values("date_yearMonth",ascending=True)
data = data[data.date_year>2015]

fig = go.Figure()

fig.add_trace(go.Scatter(x=data['date_yearMonth'], y=data['DATE_ORIGINE'], name="Montreral",
                             line=dict(color=px.colors.sequential.Plasma[0], width=2)))
fig.show()

### Is there a difference by boroughs ?

In [44]:
boroughs_top5 =  df_remorquages_mtl.groupby("ARRONDISSEMENT_ORIGINE").count().sort_values("DATE_ORIGINE",ascending=False).head(5).index


data = df_remorquages_mtl[df_remorquages_mtl["ARRONDISSEMENT_ORIGINE"].isin(boroughs_top5)]
data = data[data.date_year>2015]
data = data.groupby(["ARRONDISSEMENT_ORIGINE","date"]).count().sort_values("date",ascending=True).reset_index()

fig = go.Figure()
ci=0
for borough in boroughs_top5:
    data_boroughs = data[data["ARRONDISSEMENT_ORIGINE"]== borough]
    fig.add_trace(go.Scatter(x=data_boroughs['date'], y=data_boroughs['DATE_ORIGINE'], name=borough,
                             line=dict(color=px.colors.sequential.Plasma[ci], width=2)))
    ci= (ci+1) %8
fig.show()

###  Let's look at a more granluar period 

In [85]:
data = df_remorquages_mtl.groupby(["date_year","date"])\
                        .count()\
                        .sort_values("DATE_ORIGINE",ascending=True).reset_index()

data = data.sort_values("date",ascending=True)
data = data[data.date_year>2015]

fig = go.Figure()

fig.add_trace(go.Scatter(x=data['date'], y=data['DATE_ORIGINE'], name="Montreral",
                             line=dict(color=px.colors.sequential.Plasma[0], width=2)))

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
    )
fig.show()